In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations
import pandas as pd
from tensorflow.keras import layers, models


In [6]:
# Load the CSV files containing image labels
train_labels = pd.read_csv('/content/train.csv')
test_labels = pd.read_csv('/content/test.csv')

# Display the total number of images and classes
total_train_images = len(train_labels)
total_test_images = len(test_labels)
num_classes = len(train_labels['label'].unique())

print(f'Total training images: {total_train_images}')
print(f'Total testing images: {total_test_images}')
print(f'Number of classes: {num_classes}')
batch_size=32

Total training images: 738
Total testing images: 185
Number of classes: 2


In [7]:
train_datagen = ImageDataGenerator(
rescale=1./255,
zoom_range=0.2,
horizontal_flip=True,
validation_split=0.2)

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

In [9]:
#x_train = train_datagen.flow_from_directory()
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/train',
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode= 'categorical',
                                              subset='training')

Found 601 images belonging to 2 classes.


In [10]:
train_set.class_indices

{'bleached': 0, 'healthy': 1}

In [11]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/test',
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical',
                                            subset='validation')

Found 36 images belonging to 2 classes.


#**VGG16**

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D,Flatten,Dense

In [13]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [14]:
vgg = VGG16(weights='imagenet',include_top = False,input_shape = (299,299,3))

58889256/58889256 [==============================] - 3s 0us/step


In [15]:
for layer in vgg.layers:
  print(layer)

In [16]:
len(vgg.layers)

19

In [17]:
# Freeze the layers of the pre-trained model
for layer in vgg.layers:
    layer.trainable = False

# Create a new model using the VGG16 base
vggmodel = models.Sequential()
vggmodel.add(vgg)

# Flatten the output and add dense layers for classification
vggmodel.add(layers.Flatten())
vggmodel.add(layers.Dense(256, activation='relu'))
vggmodel.add(layers.Dropout(0.5))
vggmodel.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
vggmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
vggmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 9, 9, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 256)               10617088  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 25332290 (96.64 MB)
Trainable params: 10617602 (40.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [18]:
epochs = 10

history = vggmodel.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)


Epoch 1/10
18/18 [==============================] - 227s 12s/step - loss: 1.9309 - accuracy: 0.5852 - val_loss: 0.5904 - val_accuracy: 0.6250
Epoch 2/10
18/18 [==============================] - 16s 888ms/step - loss: 0.6249 - accuracy: 0.6678 - val_loss: 0.5926 - val_accuracy: 0.6875
Epoch 3/10
18/18 [==============================] - 17s 919ms/step - loss: 0.5591 - accuracy: 0.7170 - val_loss: 0.6045 - val_accuracy: 0.6562
Epoch 4/10
18/18 [==============================] - 17s 966ms/step - loss: 0.5431 - accuracy: 0.7311 - val_loss: 0.5734 - val_accuracy: 0.6875
Epoch 5/10
18/18 [==============================] - 18s 989ms/step - loss: 0.5087 - accuracy: 0.7452 - val_loss: 0.5552 - val_accuracy: 0.7188
Epoch 6/10
18/18 [==============================] - 16s 900ms/step - loss: 0.4507 - accuracy: 0.7909 - val_loss: 0.5041 - val_accuracy: 0.6875
Epoch 7/10
18/18 [==============================] - 18s 986ms/step - loss: 0.4955 - accuracy: 0.7504 - val_loss: 0.5255 - val_accuracy: 0.6875


In [27]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

vggmodel.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [28]:
labels = ['bleached','healthy']

In [29]:
import numpy as np
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [30]:
import numpy as np
img = '/content/drive/MyDrive/test/healthy/14307809941_27b4257449_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = vggmodel.predict(np.array([x]))
preds

1/1 [==============================] - 0s 238ms/step


array([[5.6030145e-15, 1.0000000e+00]], dtype=float32)

In [31]:
labels[np.argmax(preds)]

'healthy'

In [32]:
import numpy as np
img = '/content/drive/MyDrive/test/bleached/14633816529_af934e0aa8_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = vggmodel.predict(np.array([x]))
preds1

1/1 [==============================] - 0s 29ms/step


array([[1.8775461e-17, 1.0000000e+00]], dtype=float32)

In [33]:
labels[np.argmax(preds1)]

'healthy'

In [34]:
vggmodel.save('waves_img.h5')

In [35]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# # Define image dimensions
# IMG_HEIGHT = 224
# IMG_WIDTH = 224
# BATCH_SIZE = 32

# # Data generators with augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     validation_split=0.2
# )

# train_generator = train_datagen.flow_from_directory(
#     '/content/drive/MyDrive/train',
#     target_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     class_mode='binary',  # or 'categorical' if you have more than 2 classes
#     subset='training'
# )

# validation_generator = train_datagen.flow_from_directory(
#     '/content/drive/MyDrive/test',
#     target_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     class_mode='binary',  # or 'categorical' if you have more than 2 classes
#     subset='validation'
# )

# # Define the model (example using VGG16)
# vgg_model = tf.keras.applications.VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')

# # Unfreeze the top layers of the VGG model for fine-tuning
# for layer in vgg_model.layers[-4:]:
#     layer.trainable = True

# model = tf.keras.Sequential([
#     vgg_model,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1, activation='sigmoid')  # or more units for multiple classes with 'softmax'
# ])

# # Learning rate scheduler
# def lr_schedule(epoch, lr):
#     if epoch > 10:
#         lr = lr * tf.math.exp(-0.1)
#     return lr

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# # Callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
# lr_scheduler = LearningRateScheduler(lr_schedule)

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     epochs=20,  # Reduced number of epochs
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // BATCH_SIZE,
#     callbacks=[early_stopping, checkpoint, lr_scheduler]
# )

# # Evaluate the model on the validation data



#**Resnet**


In [36]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers

# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Create a new model using the ResNet50 base
resnet_model = models.Sequential()
resnet_model.add(resnet)

# Flatten the output and add dense layers for classification
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(256, activation='relu'))
resnet_model.add(layers.Dropout(0.5))
resnet_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
resnet_model.summary()


94765736/94765736 [==============================] - 4s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 10, 10, 2048)      23587712  
                                                                 
 flatten_1 (Flatten)         (None, 204800)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               52429056  
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 76017282 (289.98 MB)
Trainable params: 52429570 (200.00 MB)
Non-trainable params: 23587712 (89.98 MB)
_____

In [37]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= resnet_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)


Epoch 1/10
18/18 [==============================] - 27s 1s/step - loss: 10.8581 - accuracy: 0.5026 - val_loss: 0.8497 - val_accuracy: 0.5312
Epoch 2/10
18/18 [==============================] - 17s 972ms/step - loss: 1.0101 - accuracy: 0.5272 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
18/18 [==============================] - 16s 875ms/step - loss: 0.7763 - accuracy: 0.5220 - val_loss: 0.6928 - val_accuracy: 0.6250
Epoch 4/10
18/18 [==============================] - 16s 888ms/step - loss: 0.6931 - accuracy: 0.5132 - val_loss: 0.6930 - val_accuracy: 0.5625
Epoch 5/10
18/18 [==============================] - 17s 960ms/step - loss: 0.6931 - accuracy: 0.5202 - val_loss: 0.6929 - val_accuracy: 0.5625
Epoch 6/10
18/18 [==============================] - 16s 911ms/step - loss: 0.6931 - accuracy: 0.5185 - val_loss: 0.6929 - val_accuracy: 0.5312
Epoch 7/10
18/18 [==============================] - 17s 971ms/step - loss: 0.6930 - accuracy: 0.5202 - val_loss: 0.6926 - val_accuracy: 0.5625
E

In [38]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

resnet_model.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [39]:
labels = ['bleached','healthy']

In [44]:
img = '/content/drive/MyDrive/test/healthy/16426224025_f750fb7e32_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = resnet_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 0s 22ms/step


array([0.19680645, 0.3545633 ], dtype=float32)

In [45]:
labels[np.argmax(preds)]

'healthy'

In [49]:
img = '/content/drive/MyDrive/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1= resnet_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 181ms/step


array([0.18538068, 0.19686073], dtype=float32)

In [50]:
labels[np.argmax(preds1)]

'healthy'

#**Inception**

In [51]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import models, layers

# Load the pre-trained InceptionV3 model
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in inception.layers:
    layer.trainable = False

# Create a new model using the InceptionV3 base
inception_model = models.Sequential()
inception_model.add(inception)

# Add a Global Average Pooling layer
inception_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
inception_model.add(layers.Dense(256, activation='relu'))
inception_model.add(layers.Dropout(0.5))
inception_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
inception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
inception_model.summary()


87910968/87910968 [==============================] - 3s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 514       
                                                                 
Total params: 22327842 (85.17 MB)
Trainable params: 525

In [52]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= inception_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 30s 1s/step - loss: 0.7325 - accuracy: 0.5852 - val_loss: 0.5530 - val_accuracy: 0.7500
Epoch 2/10
18/18 [==============================] - 15s 857ms/step - loss: 0.5871 - accuracy: 0.6766 - val_loss: 0.5069 - val_accuracy: 0.7500
Epoch 3/10
18/18 [==============================] - 15s 861ms/step - loss: 0.5599 - accuracy: 0.7276 - val_loss: 0.5826 - val_accuracy: 0.6250
Epoch 4/10
18/18 [==============================] - 16s 887ms/step - loss: 0.5296 - accuracy: 0.7592 - val_loss: 0.5676 - val_accuracy: 0.7500
Epoch 5/10
18/18 [==============================] - 16s 876ms/step - loss: 0.5220 - accuracy: 0.7346 - val_loss: 0.5709 - val_accuracy: 0.6562
Epoch 6/10
18/18 [==============================] - 17s 943ms/step - loss: 0.5226 - accuracy: 0.7135 - val_loss: 0.4757 - val_accuracy: 0.8438
Epoch 7/10
18/18 [==============================] - 16s 888ms/step - loss: 0.4835 - accuracy: 0.7645 - val_loss: 0.4888 - val_accuracy: 0.7812
Ep

In [53]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

inception_model.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [54]:
labels = ['bleached','healthy']

In [55]:
img = '/content/drive/MyDrive/test/healthy/11181383133_411acd063f_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = inception_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 3s 3s/step


array([0.08862984, 0.13824932], dtype=float32)

In [56]:
labels[np.argmax(preds)]

'healthy'

In [58]:
img = '/content/drive/MyDrive/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = inception_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 25ms/step


array([9.999945e-01, 5.723633e-08], dtype=float32)

In [59]:
labels[np.argmax(preds1)]

'bleached'

#**Densenet**


In [60]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import models, layers

# Load the pre-trained DenseNet121 model
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in densenet.layers:
    layer.trainable = False

# Create a new model using the DenseNet121 base
densenet_model = models.Sequential()
densenet_model.add(densenet)

# Add a Global Average Pooling layer
densenet_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
densenet_model.add(layers.Dense(256, activation='relu'))
densenet_model.add(layers.Dropout(0.5))
densenet_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
densenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
densenet_model.summary()


29084464/29084464 [==============================] - 2s 0us/step
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 9, 9, 1024)        7037504   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_6 (Dense)             (None, 256)               262400    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 514       
                                                                 
Total params: 7300418 (27.85 MB)
Trainable params: 2629

In [61]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= densenet_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 39s 1s/step - loss: 0.6469 - accuracy: 0.6555 - val_loss: 0.6380 - val_accuracy: 0.6562
Epoch 2/10
18/18 [==============================] - 16s 869ms/step - loss: 0.5510 - accuracy: 0.7258 - val_loss: 0.5677 - val_accuracy: 0.7500
Epoch 3/10
18/18 [==============================] - 17s 899ms/step - loss: 0.4954 - accuracy: 0.7610 - val_loss: 0.5721 - val_accuracy: 0.6875
Epoch 4/10
18/18 [==============================] - 16s 877ms/step - loss: 0.4635 - accuracy: 0.8137 - val_loss: 0.4976 - val_accuracy: 0.7188
Epoch 5/10
18/18 [==============================] - 16s 879ms/step - loss: 0.4449 - accuracy: 0.7768 - val_loss: 0.5479 - val_accuracy: 0.8125
Epoch 6/10
18/18 [==============================] - 16s 864ms/step - loss: 0.4366 - accuracy: 0.7856 - val_loss: 0.5005 - val_accuracy: 0.7188
Epoch 7/10
18/18 [==============================] - 16s 877ms/step - loss: 0.4236 - accuracy: 0.7873 - val_loss: 0.5262 - val_accuracy: 0.7188
Ep

In [62]:
img = '/content/drive/MyDrive/test/healthy/11181383133_411acd063f_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = densenet_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 4s 4s/step


array([0.00666854, 0.8683572 ], dtype=float32)

In [63]:
labels[np.argmax(preds)]

'healthy'

In [66]:
img = '/content/drive/MyDrive/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = densenet_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 33ms/step


array([9.993168e-01, 7.462764e-05], dtype=float32)

In [67]:
labels[np.argmax(preds1)]

'bleached'

#**Xpection**

In [68]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import models, layers

# Load the pre-trained Xception model
xception = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in xception.layers:
    layer.trainable = False

# Create a new model using the Xception base
xception_model = models.Sequential()
xception_model.add(xception)

# Add a Global Average Pooling layer
xception_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
xception_model.add(layers.Dense(256, activation='relu'))
xception_model.add(layers.Dropout(0.5))
xception_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
xception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
xception_model.summary()


83683744/83683744 [==============================] - 4s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_8 (Dense)             (None, 256)               524544    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21386538 (81.58 MB)
Trainable params: 525

In [69]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= xception_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 29s 1s/step - loss: 0.6391 - accuracy: 0.6432 - val_loss: 0.6444 - val_accuracy: 0.5938
Epoch 2/10
18/18 [==============================] - 17s 946ms/step - loss: 0.5279 - accuracy: 0.7223 - val_loss: 0.6830 - val_accuracy: 0.6562
Epoch 3/10
18/18 [==============================] - 18s 966ms/step - loss: 0.5242 - accuracy: 0.7522 - val_loss: 0.7006 - val_accuracy: 0.6250
Epoch 4/10
18/18 [==============================] - 17s 967ms/step - loss: 0.4873 - accuracy: 0.7715 - val_loss: 0.6069 - val_accuracy: 0.6562
Epoch 5/10
18/18 [==============================] - 18s 1s/step - loss: 0.4703 - accuracy: 0.7891 - val_loss: 0.7219 - val_accuracy: 0.5938
Epoch 6/10
18/18 [==============================] - 17s 977ms/step - loss: 0.4619 - accuracy: 0.7768 - val_loss: 0.6721 - val_accuracy: 0.6562
Epoch 7/10
18/18 [==============================] - 17s 960ms/step - loss: 0.4266 - accuracy: 0.8120 - val_loss: 0.5425 - val_accuracy: 0.7188
Epoch

In [70]:
img = '/content/drive/MyDrive/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = xception_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 2s 2s/step


array([0.91707754, 0.46083626], dtype=float32)

In [71]:
labels[np.argmax(preds)]

'bleached'

In [72]:
img = '/content/drive/MyDrive/test/healthy/11181383133_411acd063f_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = xception_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 23ms/step


array([3.864559e-07, 9.999789e-01], dtype=float32)

In [73]:
labels[np.argmax(preds1)]

'healthy'

In [82]:
val_loss, val_accuracy = vggmodel.evaluate(test_set, steps=test_set.samples // batch_size)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

1/1 [==============================] - 1s 969ms/step - loss: 0.6468 - accuracy: 0.8125
Validation Accuracy: 81.25%


In [83]:
val_loss, val_accuracy = resnet_model.evaluate(test_set, steps=test_set.samples // batch_size)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

1/1 [==============================] - 4s 4s/step - loss: 0.6900 - accuracy: 0.5938
Validation Accuracy: 59.38%


In [84]:
val_loss, val_accuracy = inception_model.evaluate(test_set, steps=test_set.samples // batch_size)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

1/1 [==============================] - 3s 3s/step - loss: 0.4108 - accuracy: 0.8125
Validation Accuracy: 81.25%


In [85]:
val_loss, val_accuracy = densenet_model.evaluate(test_set, steps=test_set.samples // batch_size)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

1/1 [==============================] - 1s 1s/step - loss: 0.4950 - accuracy: 0.7812
Validation Accuracy: 78.12%


In [86]:
val_loss, val_accuracy = xception_model.evaluate(test_set, steps=test_set.samples // batch_size)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

1/1 [==============================] - 1s 957ms/step - loss: 0.4534 - accuracy: 0.7188
Validation Accuracy: 71.88%


In [87]:
vggmodel.save('waves_img.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
